# 1. Imports

#### General

In [7]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

In [4]:
import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score

#### PyG

In [ ]:
os.environ['TORCH'] = torch.__version__
print('torch version:', torch.__version__)

!pip3 install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip3 install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip3 install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [9]:
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, to_hetero
from torch_geometric.data import Data, HeteroData
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader
import torch_geometric.transforms as T

#### Transformers

In [5]:
!pip3 install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.7 MB/s eta 0:00:00


In [11]:
from transformers import BertTokenizer, BertModel, AutoModel, AutoTokenizer

### Device

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device:', device)

device: cpu


### Drive

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2. Dataframes

In [14]:
def get_users_and_items(users_csv, items_csv, user_naming, item_naming):
    df_users = pd.read_csv(users_csv)
    print('users\n===========')
    print('users shape:', df_users.shape)
    print('unique users:', len(df_users[user_naming].unique()))
    print('unique articles:', len(df_users[item_naming].unique()))
    print(df_users.head())
    print()

    df_items = pd.read_csv(items_csv)
    print('items\n===========')
    print('items shape:', df_items.shape)
    print('unique articles:', len(df_items[item_naming].unique()))
    print(df_items.head())
    print()

    print('article set is same in two files:', set(df_users[item_naming].unique()) == set(df_items[item_naming].unique()))
    assert set(df_users[item_naming].unique()) == set(df_items[item_naming].unique())
    assert df_items.shape[0] == df_items[item_naming].nunique()
    
    return df_users, df_items

## Adressa

### Adressa Original: NB-Bert

In [ ]:
fusers = '/content/drive/Shareddrives/NewsRec/data/adressa/impressions/common_user_item.csv'
fitems = '/content/drive/Shareddrives/NewsRec/data/adressa/content/read_articles.csv'
user_naming, item_naming = 'userId', 'id'
df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

ffeat_items = '/content/drive/Shareddrives/NewsRec/data/adressa/content/feat_tensor_original.pt'
feature_tensor = torch.load(ffeat_items, map_location=device)
print('feature tensor:', feature_tensor.shape)
assert feature_tensor.shape[0] == df_users[item_naming].nunique()

users
users shape: (1857389, 2)
unique users: 535259
unique articles: 15721
                                              userId  \
0  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
1  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
2  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
3  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
4  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   

                                         id  
0  01c74ff7ec02862bed5f861cba7e7226dfd31beb  
1  2228c8e09bd615b509f87347675f4f0f1b74439e  
2  265d0e347e08d19ab62ffe4eadd0333eaad6e57c  
3  3011c1cce4999e3c4e02bd887468d1d1d90b7807  
4  44d3e6498035638f2e1c0580332b67e0d71af45c  

items
items shape: (15721, 2)
unique articles: 15721
                                         id  \
0  6a0612e60690288a776834811004ce133f326cee   
1  b40a30877124510cf65683b6c9391d927e20f89d   
2  29b7afc1d16d34b639597cab200c9c4c96e2b69e   
3  e6877439db8ca3bde8be8f6edb741fcc22c2772c   
4  a30fec2f3e9f2e408a2

### Adressa Multiling: mBert

In [ ]:
fusers = '/content/drive/Shareddrives/NewsRec/data/adressa/impressions/common_user_item.csv'
fitems = '/content/drive/Shareddrives/NewsRec/data/adressa/content/read_articles.csv'
user_naming, item_naming = 'userId', 'id'
df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

ffeat_items = '/content/drive/Shareddrives/NewsRec/data/adressa/content/feat_tensor_multiling.pt'
feature_tensor = torch.load(ffeat_items, map_location=device)
print('feature tensor:', feature_tensor.shape)
assert feature_tensor.shape[0] == df_users[item_naming].nunique()

users
users shape: (1857389, 2)
unique users: 535259
unique articles: 15721
                                              userId  \
0  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
1  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
2  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
3  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
4  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   

                                         id  
0  01c74ff7ec02862bed5f861cba7e7226dfd31beb  
1  2228c8e09bd615b509f87347675f4f0f1b74439e  
2  265d0e347e08d19ab62ffe4eadd0333eaad6e57c  
3  3011c1cce4999e3c4e02bd887468d1d1d90b7807  
4  44d3e6498035638f2e1c0580332b67e0d71af45c  

items
items shape: (15721, 2)
unique articles: 15721
                                         id  \
0  6a0612e60690288a776834811004ce133f326cee   
1  b40a30877124510cf65683b6c9391d927e20f89d   
2  29b7afc1d16d34b639597cab200c9c4c96e2b69e   
3  e6877439db8ca3bde8be8f6edb741fcc22c2772c   
4  a30fec2f3e9f2e408a2

### Adressa Multiling: XLM

In [ ]:
fusers = '/content/drive/Shareddrives/NewsRec/data/adressa/impressions/common_user_item.csv'
fitems = '/content/drive/Shareddrives/NewsRec/data/adressa/content/read_articles.csv'
user_naming, item_naming = 'userId', 'id'
df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

ffeat_items = '/content/drive/Shareddrives/NewsRec/data/adressa/content/feat_tensor_xlm.pt'
feature_tensor = torch.load(ffeat_items, map_location=device)
print('feature tensor:', feature_tensor.shape)
assert feature_tensor.shape[0] == df_users[item_naming].nunique()

users
users shape: (1857389, 2)
unique users: 535259
unique articles: 15721
                                              userId  \
0  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
1  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
2  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
3  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
4  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   

                                         id  
0  01c74ff7ec02862bed5f861cba7e7226dfd31beb  
1  2228c8e09bd615b509f87347675f4f0f1b74439e  
2  265d0e347e08d19ab62ffe4eadd0333eaad6e57c  
3  3011c1cce4999e3c4e02bd887468d1d1d90b7807  
4  44d3e6498035638f2e1c0580332b67e0d71af45c  

items
items shape: (15721, 2)
unique articles: 15721
                                         id  \
0  6a0612e60690288a776834811004ce133f326cee   
1  b40a30877124510cf65683b6c9391d927e20f89d   
2  29b7afc1d16d34b639597cab200c9c4c96e2b69e   
3  e6877439db8ca3bde8be8f6edb741fcc22c2772c   
4  a30fec2f3e9f2e408a2

### Adressa Translated to English: Bert

In [ ]:
fusers = '/content/drive/Shareddrives/NewsRec/data/adressa/impressions/common_user_item.csv'
fitems = '/content/drive/Shareddrives/NewsRec/data/adressa/content/read_artciles_translated.csv'
user_naming, item_naming = 'userId', 'id'
df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

ffeat_items = '/content/drive/Shareddrives/NewsRec/data/adressa/content/feat_tensor_translated.pt'
feature_tensor = torch.load(ffeat_items, map_location=device)
print('feature tensor:', feature_tensor.shape)
assert feature_tensor.shape[0] == df_users[item_naming].nunique()

users
users shape: (1857389, 2)
unique users: 535259
unique articles: 15721
                                              userId  \
0  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
1  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
2  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
3  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
4  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   

                                         id  
0  01c74ff7ec02862bed5f861cba7e7226dfd31beb  
1  2228c8e09bd615b509f87347675f4f0f1b74439e  
2  265d0e347e08d19ab62ffe4eadd0333eaad6e57c  
3  3011c1cce4999e3c4e02bd887468d1d1d90b7807  
4  44d3e6498035638f2e1c0580332b67e0d71af45c  

items
items shape: (15721, 2)
unique articles: 15721
                                         id  \
0  6a0612e60690288a776834811004ce133f326cee   
1  b40a30877124510cf65683b6c9391d927e20f89d   
2  29b7afc1d16d34b639597cab200c9c4c96e2b69e   
3  e6877439db8ca3bde8be8f6edb741fcc22c2772c   
4  a30fec2f3e9f2e408a2

## CIT

### CIT Original PT: PT-Bert

In [ ]:
fusers = '/content/drive/Shareddrives/NewsRec/data/cit/common_user_item_PT.csv'
fitems = '/content/drive/Shareddrives/NewsRec/data/cit/read_articles_PT.csv'
user_naming, item_naming = 'personId', 'contentId'
df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

ffeat_items = '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_PT_original.pt'
feature_tensor = torch.load(ffeat_items, map_location=device)
print('feature tensor:', feature_tensor.shape)
assert feature_tensor.shape[0] == df_users[item_naming].nunique()

users
users shape: (14417, 4)
unique users: 1619
unique articles: 821
             contentId             personId  \
0   310515487419366995 -1130272294246983140   
1   310515487419366995   344280948527967603   
2   310515487419366995 -8763398617720485024   
3 -8864073373672512525  3609194402293569455   
4   310515487419366995  3609194402293569455   

                                                 url lang  
0  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  
1  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  
2  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  
3  http://www.startupsstars.com/2016/06/video-int...   pt  
4  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  

items
items shape: (821, 4)
unique articles: 821
             contentId                                              title  \
0 -9157338616628196758  Situação financeira ruim de varejistas pressio...   
1 -5917314377186856799  Artigos e Palestras - Programa Agricultura de ...   
2  615

### CIT Multiling: mBert

In [ ]:
fusers = '/content/drive/Shareddrives/NewsRec/data/cit/common_user_item_PT.csv'
fitems = '/content/drive/Shareddrives/NewsRec/data/cit/read_articles_PT.csv'
user_naming, item_naming = 'personId', 'contentId'
df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

ffeat_items = '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_PT_multiling.pt'
feature_tensor = torch.load(ffeat_items, map_location=device)
print('feature tensor:', feature_tensor.shape)
assert feature_tensor.shape[0] == df_users[item_naming].nunique()

users
users shape: (14417, 4)
unique users: 1619
unique articles: 821
             contentId             personId  \
0   310515487419366995 -1130272294246983140   
1   310515487419366995   344280948527967603   
2   310515487419366995 -8763398617720485024   
3 -8864073373672512525  3609194402293569455   
4   310515487419366995  3609194402293569455   

                                                 url lang  
0  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  
1  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  
2  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  
3  http://www.startupsstars.com/2016/06/video-int...   pt  
4  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  

items
items shape: (821, 4)
unique articles: 821
             contentId                                              title  \
0 -9157338616628196758  Situação financeira ruim de varejistas pressio...   
1 -5917314377186856799  Artigos e Palestras - Programa Agricultura de ...   
2  615

### CIT Multiling: XLM

In [ ]:
fusers = '/content/drive/Shareddrives/NewsRec/data/cit/common_user_item_PT.csv'
fitems = '/content/drive/Shareddrives/NewsRec/data/cit/read_articles_PT.csv'
user_naming, item_naming = 'personId', 'contentId'
df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

ffeat_items = '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_PT_xlm.pt'
feature_tensor = torch.load(ffeat_items, map_location=device)
print('feature tensor:', feature_tensor.shape)
assert feature_tensor.shape[0] == df_users[item_naming].nunique()

users
users shape: (14417, 4)
unique users: 1619
unique articles: 821
             contentId             personId  \
0   310515487419366995 -1130272294246983140   
1   310515487419366995   344280948527967603   
2   310515487419366995 -8763398617720485024   
3 -8864073373672512525  3609194402293569455   
4   310515487419366995  3609194402293569455   

                                                 url lang  
0  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  
1  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  
2  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  
3  http://www.startupsstars.com/2016/06/video-int...   pt  
4  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  

items
items shape: (821, 4)
unique articles: 821
             contentId                                              title  \
0 -9157338616628196758  Situação financeira ruim de varejistas pressio...   
1 -5917314377186856799  Artigos e Palestras - Programa Agricultura de ...   
2  615

### CIT Translated to English: Bert

In [ ]:
fusers = '/content/drive/Shareddrives/NewsRec/data/cit/common_user_item_PT.csv'
fitems = '/content/drive/Shareddrives/NewsRec/data/cit/read_artciles_PT_translated.csv'
user_naming, item_naming = 'personId', 'url'
df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

ffeat_items = '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_PT_translated.pt'
feature_tensor = torch.load(ffeat_items, map_location=device)
print('feature tensor:', feature_tensor.shape)
assert feature_tensor.shape[0] == df_users[item_naming].nunique()

users
users shape: (14417, 4)
unique users: 1619
unique articles: 821
             contentId             personId  \
0   310515487419366995 -1130272294246983140   
1   310515487419366995   344280948527967603   
2   310515487419366995 -8763398617720485024   
3 -8864073373672512525  3609194402293569455   
4   310515487419366995  3609194402293569455   

                                                 url lang  
0  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  
1  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  
2  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  
3  http://www.startupsstars.com/2016/06/video-int...   pt  
4  http://blog.runrun.it/erros-de-portugues-em-e-...   pt  

items
items shape: (821, 3)
unique articles: 821
      contentId                                              title  \
0 -9.160000e+18  Bad financial situation of retailers puts pres...   
1 -5.920000e+18  Articles and Lectures - SENAR Precision Agricu...   
2  6.160000e+18            

### CIT Original EN: Bert

In [ ]:
fusers = '/content/drive/Shareddrives/NewsRec/data/cit/common_user_item_EN.csv'
fitems = '/content/drive/Shareddrives/NewsRec/data/cit/read_articles_EN.csv'
user_naming, item_naming = 'personId', 'contentId'
df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

ffeat_items = '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_EN.pt'
feature_tensor = torch.load(ffeat_items, map_location=device)
print('feature tensor:', feature_tensor.shape)
assert feature_tensor.shape[0] == df_users[item_naming].nunique()

users
users shape: (26206, 4)
unique users: 1644
unique articles: 2154
             contentId             personId  \
0 -3499919498720038879 -8845298781299428018   
1  8890720798209849691 -1032019229384696495   
2 -7820640624231356730  -445337111692715325   
3 -1492913151930215984  4254153380739593270   
4  3064370296170038610  3609194402293569455   

                                                 url lang  
0             http://techcrunch.com/2016/06/07/hiri/   en  
1  https://www.nngroup.com/articles/top-intranet-...   en  
2  http://www.fastcompany.com/3058876/your-most-p...   en  
3  https://developer.chrome.com/devtools/docs/con...   en  
4  https://www.linkedin.com/pulse/google-amazon-u...   en  

items
items shape: (2154, 4)
unique articles: 2154
             contentId                                              title  \
0 -6451309518266745024  Ethereum, a Virtual Currency, Enables Transact...   
1 -4110354420726924665  Ethereum, a Virtual Currency, Enables Transact...   
2 -

### CIT Mixed: mBert

In [ ]:
fusers = '/content/drive/Shareddrives/NewsRec/data/cit/common_user_item.csv'
fitems = '/content/drive/Shareddrives/NewsRec/data/cit/read_articles.csv'
user_naming, item_naming = 'personId', 'contentId'
df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

ffeat_items = '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_mbert.pt'
feature_tensor = torch.load(ffeat_items, map_location=device)
print('feature tensor:', feature_tensor.shape)
assert feature_tensor.shape[0] == df_users[item_naming].nunique()

users
users shape: (40623, 4)
unique users: 1894
unique articles: 2975
             contentId             personId  \
0 -3499919498720038879 -8845298781299428018   
1  8890720798209849691 -1032019229384696495   
2 -7820640624231356730  -445337111692715325   
3 -1492913151930215984  4254153380739593270   
4  3064370296170038610  3609194402293569455   

                                                 url lang  
0             http://techcrunch.com/2016/06/07/hiri/   en  
1  https://www.nngroup.com/articles/top-intranet-...   en  
2  http://www.fastcompany.com/3058876/your-most-p...   en  
3  https://developer.chrome.com/devtools/docs/con...   en  
4  https://www.linkedin.com/pulse/google-amazon-u...   en  

items
items shape: (2975, 4)
unique articles: 2975
             contentId                                              title  \
0 -6451309518266745024  Ethereum, a Virtual Currency, Enables Transact...   
1 -4110354420726924665  Ethereum, a Virtual Currency, Enables Transact...   
2 -

### CIT Mixed: XLM

In [ ]:
fusers = '/content/drive/Shareddrives/NewsRec/data/cit/common_user_item.csv'
fitems = '/content/drive/Shareddrives/NewsRec/data/cit/read_articles.csv'
user_naming, item_naming = 'personId', 'contentId'
df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

ffeat_items = '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_xlm.pt'
feature_tensor = torch.load(ffeat_items, map_location=device)
print('feature tensor:', feature_tensor.shape)
assert feature_tensor.shape[0] == df_users[item_naming].nunique()

users
users shape: (40623, 4)
unique users: 1894
unique articles: 2975
             contentId             personId  \
0 -3499919498720038879 -8845298781299428018   
1  8890720798209849691 -1032019229384696495   
2 -7820640624231356730  -445337111692715325   
3 -1492913151930215984  4254153380739593270   
4  3064370296170038610  3609194402293569455   

                                                 url lang  
0             http://techcrunch.com/2016/06/07/hiri/   en  
1  https://www.nngroup.com/articles/top-intranet-...   en  
2  http://www.fastcompany.com/3058876/your-most-p...   en  
3  https://developer.chrome.com/devtools/docs/con...   en  
4  https://www.linkedin.com/pulse/google-amazon-u...   en  

items
items shape: (2975, 4)
unique articles: 2975
             contentId                                              title  \
0 -6451309518266745024  Ethereum, a Virtual Currency, Enables Transact...   
1 -4110354420726924665  Ethereum, a Virtual Currency, Enables Transact...   
2 -

### CIT Train on Orig EN only and Test on Mixed

Train on Orig EN first

Then test on the following Mix

In [ ]:
fusers = '/content/drive/Shareddrives/NewsRec/data/cit/common_user_item_MIX.csv'
fitems = '/content/drive/Shareddrives/NewsRec/data/cit/read_artciles_MIX_EN_TR.csv'
user_naming, item_naming = 'personId', 'url'
df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

ffeat_items = '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_MIX.pt'
feature_tensor = torch.load(ffeat_items, map_location=device)
print('feature tensor:', feature_tensor.shape)
assert feature_tensor.shape[0] == df_users[item_naming].nunique()

### CIT Train with Mixed (Orig EN + Translated PT) and Test on Mixed

In [ ]:
fusers = '/content/drive/Shareddrives/NewsRec/data/cit/common_user_item_MIX.csv'
fitems = '/content/drive/Shareddrives/NewsRec/data/cit/read_artciles_MIX_EN_TR.csv'
user_naming, item_naming = 'personId', 'url'
df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

ffeat_items = '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_MIX.pt'
feature_tensor = torch.load(ffeat_items, map_location=device)
print('feature tensor:', feature_tensor.shape)
assert feature_tensor.shape[0] == df_users[item_naming].nunique()

## MIND

In [ ]:
i = 0
fusers = f'/content/drive/Shareddrives/NewsRec/data/mind/MINDlarge_train/common_user_item_{i}.csv'
fitems = f'/content/drive/Shareddrives/NewsRec/data/mind/MINDlarge_train/read_article_titles_{i}.csv'
user_naming, item_naming = 'User_ID', 'News_ID'
df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

ffeat_items = f'/content/drive/Shareddrives/NewsRec/data/mind/MINDlarge_train/feat_tensor_{i}.pt'
feature_tensor = torch.load(ffeat_items, map_location=device)
print('feature tensor:', feature_tensor.shape)
assert feature_tensor.shape[0] == df_users[item_naming].nunique()

users
users shape: (4658157, 2)
unique users: 155855
unique articles: 67147
   User_ID  News_ID
0  U321454  N128643
1  U578952  N122359
2  U578952  N110096
3  U578952   N20583
4  U578952  N128736

items
items shape: (67147, 2)
unique articles: 67147
  News_ID                                              title
0  N88753  The Brands Queen Elizabeth, Prince Charles, an...
1  N45436    Walmart Slashes Prices on Last-Generation iPads
2  N23144                      50 Worst Habits For Belly Fat
3  N86255  Dispose of unwanted prescription drugs during ...
4  N93187  The Cost of Trump's Aid Freeze in the Trenches...

article set is same: True
feature tensor: torch.Size([67147, 768])


# Run PLM

In [ ]:
def load_plm_model(plm_model_name):
    print('Hi, loading:', plm_model_name)
    plm_model = AutoModel.from_pretrained(plm_model_name).to(device)
    plm_tokenizer = AutoTokenizer.from_pretrained(plm_model_name)
    return plm_model, plm_tokenizer

def get_plm_embeddings(plm_model, plm_tokenizer, df_items, title_naming, feat_tensor_file):
    print('df_items shape:', df_items.shape)

    encoded_inputs = plm_tokenizer(list(df_items[title_naming]), max_length=50, padding=True, truncation=True, return_tensors="pt")
    input_ids = encoded_inputs['input_ids'].to(device)
    print('input_ids shape:', input_ids.shape)

    feat_vectors = []
    i = 0 
    while i < len(input_ids):
        if i+512 >= len(input_ids):
            encoded_inputs_batch = input_ids[i:, :]
        else:
            encoded_inputs_batch = input_ids[i:i+512, :]
        with torch.no_grad():
            out = plm_model(encoded_inputs_batch)[1]
        feat_vectors.append(out)
        print(f'samples done: {i} / {len(input_ids)}')
        i += 512
    
    feat_tensor = torch.cat(feat_vectors, 0)
    print('feat_tensor shape:', feat_tensor.shape)

    torch.save(feat_tensor, feat_tensor_file)

    return feat_tensor

In [ ]:
plm_models = {
    'mbert': 'bert-base-multilingual-cased',
    'bert': 'bert-base-cased',
    'nb-bert': 'NbAiLab/nb-bert-base',
    'pt-bert': 'neuralmind/bert-base-portuguese-cased',
    'xlm': 'microsoft/infoxlm-base',
    'gpt2': 'gpt2'
}

In [ ]:
plm_model_name = plm_models['xlm']
plm_model, plm_tokenizer = load_plm_model(plm_model_name)

Hi, loading: microsoft/infoxlm-base


Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/942M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/infoxlm-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
title_naming = 'title'
feat_tensor_file = '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_xlm.pt'
ften = get_plm_embeddings(plm_model, plm_tokenizer, df_items, title_naming, feat_tensor_file)

df_items shape: (2975, 4)
input_ids shape: torch.Size([2975, 47])
samples done: 0 / 2975
samples done: 512 / 2975
samples done: 1024 / 2975
samples done: 1536 / 2975
samples done: 2048 / 2975
samples done: 2560 / 2975
feat_tensor shape: torch.Size([2975, 768])


# 3. Graph Construction

We construct graph from the user-item interaction.

In [ ]:
user_has_features = True
user_feature_size = 16          # 32
item_feature_init = 'plm'       # 'plm', 'zero', 'random'
item_feature_size = 768         # feature_tensor.size()[1]
random_edges = False

#### Reload

In [ ]:
# Create a mapping from unique user indices to range [0, num_user_nodes):
unique_user_id = df_users[user_naming].unique()
unique_user_id = pd.DataFrame(data={
    'userId': unique_user_id,
    'mappedID': pd.RangeIndex(len(unique_user_id)),
})
print("Mapping of user IDs to consecutive values:")
print("==========================================")
print(unique_user_id.head())
print()

Mapping of user IDs to consecutive values:
                userId  mappedID
0 -1130272294246983140         0
1   344280948527967603         1
2 -8763398617720485024         2
3  3609194402293569455         3
4  1908339160857512799         4



In [ ]:
# Create a mapping from unique item indices to range [0, num_item_nodes):
assert not any(df_items[item_naming].values != df_items[item_naming].unique())
unique_item_id = df_items[item_naming]
unique_item_id = pd.DataFrame(data={
    'itemId': unique_item_id,
    'mappedID': pd.RangeIndex(len(unique_item_id)),
})
print("Mapping of item IDs to consecutive values:")
print("===========================================")
print(unique_item_id.head())

Mapping of item IDs to consecutive values:
                itemId  mappedID
0 -9157338616628196758         0
1 -5917314377186856799         1
2  6157037646878010131         2
3 -1672166631728511207         3
4  5714314286511882372         4


In [ ]:
# Perform merge to obtain the edges from users and items:
ratings_user_id = pd.merge(df_users[user_naming], unique_user_id,
                            left_on=user_naming, right_on='userId', how='left')
ratings_user_id = torch.from_numpy(ratings_user_id['mappedID'].values)
ratings_item_id = pd.merge(df_users[item_naming], unique_item_id,
                            left_on=item_naming, right_on='itemId', how='left')
ratings_item_id = torch.from_numpy(ratings_item_id['mappedID'].values)

Random edges result in 0.5 AUC and 0.0 F1 score.

In [ ]:
# Random edges
if random_edges:
  ratings_user_id = torch.from_numpy(np.random.randint(low=0, high=len(unique_user_id)-1, size=len(unique_user_id)))
  ratings_item_id = torch.from_numpy(np.random.randint(low=0, high=len(unique_item_id)-1, size=len(unique_user_id)))

In [ ]:
# With this, we are ready to construct our `edge_index` in COO format
# following PyG semantics:
edge_index_coo_user_to_item = torch.stack([ratings_user_id, ratings_item_id], dim=0)

print()
print("Final edge indices pointing from users to items:")
print("=================================================")
print(edge_index_coo_user_to_item)


Final edge indices pointing from users to items:
tensor([[   0,    1,    2,  ...,   46, 1405, 1468],
        [ 260,  260,  260,  ...,  783,  618,  782]])


In [ ]:
data = HeteroData()

# Save node indices:
data["user"].node_id = torch.arange(len(unique_user_id))
data["item"].node_id = torch.arange(len(unique_item_id))

# Add the node features and edge indices:
if item_feature_init == 'plm':
  data["item"].x = feature_tensor  # DONE
  if user_has_features:
  # only random init for user features
    data["user"].x = torch.Tensor(np.random.random((len(unique_user_id), user_feature_size)))
    print('user features shape:', data["user"].x.shape)
elif item_feature_init == 'random':
  data["item"].x = torch.Tensor(np.random.random((len(unique_item_id), item_feature_size)))
elif item_feature_init == 'zero':
  data["item"].x = torch.Tensor(np.zeros((len(unique_item_id), item_feature_size)))
else:
  raise 'Need item feature init type: plm, random, zero'

print('item features shape:', data["item"].x.shape)
data["user", "rates", "item"].edge_index = edge_index_coo_user_to_item  # DONE

# We also need to make sure to add the reverse edges from items to users
# in order to let a GNN be able to pass messages in both directions.
# We can leverage the `T.ToUndirected()` transform for this from PyG:
data = T.ToUndirected()(data)
del data['item', 'rev_rates', 'user'].edge_label  # Remove "reverse" label.

print(data)
print("=============================")
print('user num_nodes:', data["user"].num_nodes)
print('user num_features:', data["user"].num_features)
print('item num_nodes:', data["item"].num_nodes)
print('item num_features:', data["item"].num_features)
print('num_edges user->item:', data["user", "rates", "item"].num_edges)
print('num_edges item->user:', data["item", "rev_rates", "user"].num_edges)

user features shape: torch.Size([1619, 16])
item features shape: torch.Size([821, 768])
HeteroData(
  user={
    node_id=[1619],
    x=[1619, 16]
  },
  item={
    node_id=[821],
    x=[821, 768]
  },
  (user, rates, item)={ edge_index=[2, 14417] },
  (item, rev_rates, user)={ edge_index=[2, 14417] }
)
user num_nodes: 1619
user num_features: 16
item num_nodes: 821
item num_features: 768
num_edges user->item: 14417
num_edges item->user: 14417


user num_features: 0
shouldn't be randomly intialized node features???

# 4. Data Loaders

In [ ]:
neg_sampling_ratio = 1.0        # hyper: 1, 2
batch_size = 128
num_neighbors = [10, 5]         # hyper: [10, 5], [10, 10], [20, 10], [20, 20]

#### Reload

In [ ]:
# For this, we first split the set of edges into
# training (80%), validation (10%), and testing edges (10%).
# Across the training edges, we use 70% of edges for message passing,
# and 30% of edges for supervision.
# We further want to generate fixed negative edges for evaluation with a ratio of 2:1.
# Negative edges during training will be generated on-the-fly.
# We can leverage the `RandomLinkSplit()` transform for this from PyG:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=neg_sampling_ratio,
    add_negative_train_samples=False,       # negative samples generated on-the-fly
    edge_types=("user", "rates", "item"),
    rev_edge_types=("item", "rev_rates", "user"), 
)


train_data, val_data, test_data = transform(data)
print("Training data:")
print("==============")
print(train_data)
print()
print("Validation data:")
print("================")
print(val_data)
print()
print("Test data:")
print("================")
print(test_data)

Training data:
HeteroData(
  user={
    node_id=[1619],
    x=[1619, 16]
  },
  item={
    node_id=[821],
    x=[821, 768]
  },
  (user, rates, item)={
    edge_index=[2, 8075],
    edge_label=[3460],
    edge_label_index=[2, 3460]
  },
  (item, rev_rates, user)={ edge_index=[2, 8075] }
)

Validation data:
HeteroData(
  user={
    node_id=[1619],
    x=[1619, 16]
  },
  item={
    node_id=[821],
    x=[821, 768]
  },
  (user, rates, item)={
    edge_index=[2, 11535],
    edge_label=[2882],
    edge_label_index=[2, 2882]
  },
  (item, rev_rates, user)={ edge_index=[2, 11535] }
)

Test data:
HeteroData(
  user={
    node_id=[1619],
    x=[1619, 16]
  },
  item={
    node_id=[821],
    x=[821, 768]
  },
  (user, rates, item)={
    edge_index=[2, 12976],
    edge_label=[2882],
    edge_label_index=[2, 2882]
  },
  (item, rev_rates, user)={ edge_index=[2, 12976] }
)


In [ ]:
print('Train data:')
print('=============')
print('present num_edges user->item:', train_data["user", "rates", "item"].num_edges)
print('to be predicted positive num_edges user->item:', train_data["user", "rates", "item"].edge_label_index.shape[1])
print('to be predicted edge classes:', torch.unique(train_data["user", "rates", "item"].edge_label))
print('negative edge classes [0.] would be generated during training on-the-fly')

Train data:
present num_edges user->item: 8075
to be predicted positive num_edges user->item: 3460
to be predicted edge classes: tensor([1.])
negative edge classes [0.] would be generated during training on-the-fly


In [ ]:
# Training data:
# edge_index=[2, 1040140],      - number of edges to construct graph, 70% of edges for message passing,
#                               - edges already present in the graph
# edge_label=[445773],          - number of edges for training, 30% of edges for supervision.
#                               - labels of missing edges, [0, 1] - 0 for negative edge, 1 for positive
# edge_label_index=[2, 445773]  - edges that are absent in the graph for training 

So, we see that all the nodes are present both in training and validation data!

As well as all the item_node features.

Only the edge_index changes: which users connected to which items. Some of them are removed for train and val. 

We are now ready to create a mini-batch loader that will generate subgraphs that can be used as input into our GNN. While this step is not strictly necessary for small-scale graphs, it is absolutely necessary to apply GNNs on larger graphs that do not fit onto GPU memory otherwise. Here, we make use of the loader.LinkNeighborLoader which samples multiple hops from both ends of a link and creates a subgraph from it. Here, edge_label_index serves as the "seed links" to start sampling from.

In [ ]:
# In the first hop, we sample at most 10 neighbors.
# In the second hop, we sample at most 5 neighbors.
# In addition, during training, we want to sample negative edges on-the-fly with
# a ratio of 2:1.
# We can make use of the `loader.LinkNeighborLoader` from PyG:
# Define seed edges:
edge_label_index = train_data["user", "rates", "item"].edge_label_index
edge_label = train_data["user", "rates", "item"].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=num_neighbors,
    neg_sampling_ratio=neg_sampling_ratio,
    edge_label_index=(("user", "rates", "item"), edge_label_index),
    edge_label=edge_label,
    batch_size=batch_size,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled Train mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data["user", "rates", "item"].edge_label_index.size(1) == (neg_sampling_ratio + 1) * batch_size # 2 edges (1 pos + 1 neg) * batch_size
assert sampled_data["user", "rates", "item"].edge_label.min() == 0
assert sampled_data["user", "rates", "item"].edge_label.max() == 1

Sampled Train mini-batch:
HeteroData(
  user={
    node_id=[961],
    x=[961, 16]
  },
  item={
    node_id=[675],
    x=[675, 768]
  },
  (user, rates, item)={
    edge_index=[2, 3072],
    edge_label=[256],
    edge_label_index=[2, 256],
    input_id=[128]
  },
  (item, rev_rates, user)={ edge_index=[2, 3146] }
)


In [ ]:
# Define the validation seed edges:
edge_label_index = val_data["user", "rates", "item"].edge_label_index
edge_label = val_data["user", "rates", "item"].edge_label

val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=num_neighbors,
    edge_label_index=(("user", "rates", "item"), edge_label_index),
    edge_label=edge_label,
    batch_size=int((neg_sampling_ratio + 1) * batch_size),
    shuffle=False,
)

sampled_data = next(iter(val_loader))

print("Sampled Validation mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data["user", "rates", "item"].edge_label_index.size(1) == (neg_sampling_ratio + 1) * batch_size
assert sampled_data["user", "rates", "item"].edge_label.min() >= 0
assert sampled_data["user", "rates", "item"].edge_label.max() <= 1

Sampled Validation mini-batch:
HeteroData(
  user={
    node_id=[990],
    x=[990, 16]
  },
  item={
    node_id=[679],
    x=[679, 768]
  },
  (user, rates, item)={
    edge_index=[2, 3520],
    edge_label=[256],
    edge_label_index=[2, 256],
    input_id=[256]
  },
  (item, rev_rates, user)={ edge_index=[2, 3751] }
)


In [ ]:
# Define the test seed edges:
edge_label_index = test_data["user", "rates", "item"].edge_label_index
edge_label = test_data["user", "rates", "item"].edge_label

test_loader = LinkNeighborLoader(
    data=test_data,
    num_neighbors=num_neighbors,
    edge_label_index=(("user", "rates", "item"), edge_label_index),
    edge_label=edge_label,
    batch_size=int((neg_sampling_ratio + 1) * batch_size),
    shuffle=False,
)

sampled_data = next(iter(test_loader))

print("Sampled Test mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data["user", "rates", "item"].edge_label_index.size(1) == (neg_sampling_ratio + 1) * batch_size
assert sampled_data["user", "rates", "item"].edge_label.min() >= 0
assert sampled_data["user", "rates", "item"].edge_label.max() <= 1

Sampled Test mini-batch:
HeteroData(
  user={
    node_id=[979],
    x=[979, 16]
  },
  item={
    node_id=[681],
    x=[681, 768]
  },
  (user, rates, item)={
    edge_index=[2, 3475],
    edge_label=[256],
    edge_label_index=[2, 256],
    input_id=[256]
  },
  (item, rev_rates, user)={ edge_index=[2, 3866] }
)


# 5. Model

In [ ]:
hidden_channels = 16                          # hyper: 16, 32, 64
GNN_Conv_Layer = SAGEConv                     # hyper: SAGEConv, GATConv
remove_embedding_layer = user_has_features    # False

#### Reload

We are now ready to create our heterogeneous GNN. The GNN is responsible for learning enriched node representations from the surrounding subgraphs, which can be then used to derive edge-level predictions. For defining our heterogenous GNN, we make use of nn.SAGEConv and the nn.to_hetero() function, which transforms a GNN defined on homogeneous graphs to be applied on heterogeneous ones.
In addition, we define a final link-level classifier, which simply takes both node embeddings of the link we are trying to predict, and applies a dot-product on them.
As users do not have any node-level information, we choose to learn their features jointly via a torch.nn.Embedding layer. In order to improve the expressiveness of movie features, we do the same for movie nodes, and simply add their shallow embeddings to the pre-defined genre features.

In [ ]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        if GNN_Conv_Layer == GATConv:
            self.conv1 = GNN_Conv_Layer(hidden_channels, hidden_channels, add_self_loops=False)
            self.conv2 = GNN_Conv_Layer(hidden_channels, hidden_channels, add_self_loops=False)
        else:
            self.conv1 = GNN_Conv_Layer(hidden_channels, hidden_channels)
            self.conv2 = GNN_Conv_Layer(hidden_channels, hidden_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        # Define a 2-layer GNN computation graph.
        # Use a *single* `ReLU` non-linearity in-between.
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# Our final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def forward(self, x_user: Tensor, x_item: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_user = x_user[edge_label_index[0]]    # feat[ids of users in coo]
        edge_feat_item = x_item[edge_label_index[1]]    # feat[ids of items in coo]

        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_user * edge_feat_item).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, user_fsize, item_fsize, num_user_nodes, num_item_nodes, hidden_channels, graph_metadata):
        super().__init__()
        # learn embedding matrices for users and items:
        if remove_embedding_layer:
          self.user_lin = torch.nn.Linear(user_fsize, hidden_channels)
        else:
          self.user_emb = torch.nn.Embedding(num_user_nodes, hidden_channels)
          self.item_emb = torch.nn.Embedding(num_item_nodes, hidden_channels)

        self.item_lin = torch.nn.Linear(item_fsize, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)     # Encoder
        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=graph_metadata, aggr='mean')
        self.classifier = Classifier()      # Deocder

    def forward(self, data: HeteroData) -> Tensor:
        if remove_embedding_layer:
          x_dict = {
            "user": self.user_lin(data["user"].x),
            "item": self.item_lin(data["item"].x),
          }
        else:
          x_dict = {
            "user": self.user_emb(data["user"].node_id),
            "item": self.item_lin(data["item"].x) + self.item_emb(data["item"].node_id),
          }

        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        # Encode
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        # Decode
        pred = self.classifier(
            x_dict["user"],
            x_dict["item"],
            data["user", "rates", "item"].edge_label_index,
        )

        return pred

In [ ]:
# data contains the whole graph
num_user_nodes = data["user"].num_nodes
num_item_nodes = data["item"].num_nodes
graph_metadata = data.metadata()

model = Model(user_feature_size, item_feature_size, num_user_nodes, num_item_nodes, hidden_channels, graph_metadata)
model = model.to(device)
print(f"Device: '{device}'")
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = F.binary_cross_entropy_with_logits

Device: 'cpu'
Model(
  (user_lin): Linear(in_features=32, out_features=16, bias=True)
  (item_lin): Linear(in_features=768, out_features=16, bias=True)
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (user__rates__item): SAGEConv(16, 16, aggr=mean)
      (item__rev_rates__user): SAGEConv(16, 16, aggr=mean)
    )
    (conv2): ModuleDict(
      (user__rates__item): SAGEConv(16, 16, aggr=mean)
      (item__rev_rates__user): SAGEConv(16, 16, aggr=mean)
    )
  )
  (classifier): Classifier()
)


# 6. Experiments

question is: is it enough to just translate news to english and use english recommender model?
or can use single multiling model for multiple languages?
or should we use a separate model for each language?

hypothesis: multiling is the best

In [ ]:
epochs = 1      # mind=1, adressa=2, cit=50 

#### Reload

In [ ]:
# training
train_loss = []
for epoch in range(1, epochs+1):
    model.train()
    total_loss = total_examples = 0
    for batch_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        batch_data = batch_data.to(device)
        
        pred = model(batch_data) # `forward` pass of the model
        ground_truth = batch_data['user', 'item'].edge_label
        
        loss = criterion(pred, ground_truth)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    
    epoch_loss = total_loss / total_examples
    train_loss.append(epoch_loss)
    print(f"\nEpoch: {epoch:03d}, Loss: {epoch_loss:.4f}")

In [ ]:
# validation
preds, ground_truths = [], []
model.eval()
for batch_data in tqdm.tqdm(val_loader):
    with torch.no_grad():
        batch_data = batch_data.to(device)
        preds.append(model(batch_data))
        ground_truths.append(batch_data['user', 'item'].edge_label)

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
auc = roc_auc_score(ground_truth, pred)
print()
print(f"Validation AUC: {auc:.4f}")
print(f'Neg sampling ratio:', neg_sampling_ratio)
print(f'Num neighbors:', num_neighbors)      
print(f'Hidden channels:',hidden_channels)
print(f'GNN_Conv_Layer:', GNN_Conv_Layer)


100%|██████████| 12/12 [00:00<00:00, 29.71it/s]


Validation AUC: 0.7862
Neg sampling ratio: 1.0
Num neighbors: [10, 5]
Hidden channels: 16
GNN_Conv_Layer: <class 'torch_geometric.nn.conv.sage_conv.SAGEConv'>


In [ ]:
# test
preds, ground_truths = [], []
model.eval()
for batch_data in tqdm.tqdm(test_loader):
    with torch.no_grad():
        batch_data = batch_data.to(device)
        preds.append(model(batch_data))
        ground_truths.append(batch_data['user', 'item'].edge_label)

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
auc = roc_auc_score(ground_truth, pred)
print()
print(f"Test AUC: {auc:.4f}")

100%|██████████| 12/12 [00:00<00:00, 22.04it/s]


Test AUC: 0.7870


#### Results

In [ ]:
out = torch.Tensor(pred).sigmoid().cpu().numpy()
auc = roc_auc_score(ground_truth, out)
print(f'Val AUC with sigmoid:', auc)

probs = np.rint(out)
f1_s = f1_score(ground_truth, probs)
print(f'Val F1 with sigmoid:', f1_s)
print(f'Neg sampling ratio:', neg_sampling_ratio)
print(f'Num neighbors:', num_neighbors)      
print(f'Hidden channels:',hidden_channels)
print(f'GNN_Conv_Layer:', GNN_Conv_Layer)

out = torch.Tensor(pred).sigmoid().cpu().numpy()
auc = roc_auc_score(ground_truth, out)
print(f'Test AUC with sigmoid:', auc)

probs = np.rint(out)
f1_s = f1_score(ground_truth, probs)
print(f'Test F1 with sigmoid:', f1_s)

Results for CIT Mixed Bert GATConv:
- 
Val AUC with sigmoid: 0.867049439982584
Val F1 with sigmoid: 0.6727344677483182
Validation AUC: 0.8670
Neg sampling ratio: 2.0
Num neighbors: [20, 20]
Hidden channels: 32
GNN_Conv_Layer: <class 'torch_geometric.nn.conv.gat_conv.GATConv'>
- 
Test AUC with sigmoid: 0.8572776203217435
Test F1 with sigmoid: 0.646524064171123

Results for CIT Mixed Bert SAGEConv:
- 
Val AUC with sigmoid: 0.8697367381170392
Val F1 with sigmoid: 0.6960985626283367
Validation AUC: 0.8697
Neg sampling ratio: 2.0
Num neighbors: [20, 20]
Hidden channels: 32
GNN_Conv_Layer: <class 'torch_geometric.nn.conv.sage_conv.SAGEConv'>
- 
Test AUC with sigmoid: 0.8618162086865792
Test F1 with sigmoid: 0.6903941829314963

Results for CIT Mixed XLM SAGEConv:
- 
Val AUC with sigmoid: 0.8700091346318184
Val F1 with sigmoid: 0.7060878243512974
Validation AUC: 0.8700
Neg sampling ratio: 2.0
Num neighbors: [20, 20]
Hidden channels: 16
GNN_Conv_Layer: <class 'torch_geometric.nn.conv.sage_conv.SAGEConv'>
- 
Test AUC with sigmoid: 0.8699248611077778
Test F1 with sigmoid: 0.7104747919725893

- Val AUC with sigmoid: 0.8670928040289351
Val F1 with sigmoid: 0.6989166036785085
Validation AUC: 0.8671
Neg sampling ratio: 2.0
Num neighbors: [10, 5]
Hidden channels: 16
GNN_Conv_Layer: <class 'torch_geometric.nn.conv.sage_conv.SAGEConv'>
- 
Test AUC with sigmoid: 0.8667656645723439
Test F1 with sigmoid: 0.696376720111097

Test with feature_tensor = 0 for adressa:
- Test AUC with sigmoid: 0.9906381031759737
- Test F1 with sigmoid: 0.9496406713155481

Test with feature_tensor = 0 for cit pt:
- Test AUC with sigmoid: 0.811534394006013
- Test F1 with sigmoid: 0.6191342306293198

Test with feature_tensor = random for cit pt:
- Test AUC with sigmoid: 0.8127672490140773
- Test F1 with sigmoid: 0.5971082454083627

Test with removing embed layer and adding random init user features for CIT PT XLM:
- Test AUC with sigmoid: 0.7870459686363612
- Test F1 with sigmoid: 0.7361064891846923

# Loop

##### Dict

In [ ]:
datas_dict  = {'adressa_nb':  ['/content/drive/Shareddrives/NewsRec/data/adressa/impressions/common_user_item.csv',
                              '/content/drive/Shareddrives/NewsRec/data/adressa/content/read_articles.csv',
                              '/content/drive/Shareddrives/NewsRec/data/adressa/content/feat_tensor_original.pt',
                              'userId', 'id', 5],
               'adressa_mb':  ['/content/drive/Shareddrives/NewsRec/data/adressa/impressions/common_user_item.csv',
                              '/content/drive/Shareddrives/NewsRec/data/adressa/content/read_articles.csv',
                              '/content/drive/Shareddrives/NewsRec/data/adressa/content/feat_tensor_multiling.pt',
                              'userId', 'id', 5],
               'adressa_xlm': ['/content/drive/Shareddrives/NewsRec/data/adressa/impressions/common_user_item.csv',
                              '/content/drive/Shareddrives/NewsRec/data/adressa/content/read_articles.csv',
                              '/content/drive/Shareddrives/NewsRec/data/adressa/content/feat_tensor_xlm.pt',
                              'userId', 'id', 5],
               'adressa_tr':  ['/content/drive/Shareddrives/NewsRec/data/adressa/impressions/common_user_item.csv',
                              '/content/drive/Shareddrives/NewsRec/data/adressa/content/read_artciles_translated.csv',
                              '/content/drive/Shareddrives/NewsRec/data/adressa/content/feat_tensor_translated.pt',
                              'userId', 'id', 5],
               
               'cit_pt':      ['/content/drive/Shareddrives/NewsRec/data/cit/common_user_item_PT.csv',
                              '/content/drive/Shareddrives/NewsRec/data/cit/read_articles_PT.csv',
                              '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_PT_original.pt',
                              'personId', 'contentId', 100],
               'cit_mb':      ['/content/drive/Shareddrives/NewsRec/data/cit/common_user_item_PT.csv',
                              '/content/drive/Shareddrives/NewsRec/data/cit/read_articles_PT.csv',
                              '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_PT_multiling.pt',
                              'personId', 'contentId', 100],
               'cit_xlm':     ['/content/drive/Shareddrives/NewsRec/data/cit/common_user_item_PT.csv',
                              '/content/drive/Shareddrives/NewsRec/data/cit/read_articles_PT.csv',
                              '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_PT_xlm.pt',
                              'personId', 'contentId', 100],
               'cit_tr':      ['/content/drive/Shareddrives/NewsRec/data/cit/common_user_item_PT.csv',
                              '/content/drive/Shareddrives/NewsRec/data/cit/read_artciles_PT_translated.csv',
                              '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_PT_translated.pt',
                              'personId', 'url', 100],
               'cit_en_b':    ['/content/drive/Shareddrives/NewsRec/data/cit/common_user_item_EN.csv',
                              '/content/drive/Shareddrives/NewsRec/data/cit/read_articles_EN.csv',
                              '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_EN.pt',
                              'personId', 'contentId', 100],
               'cit_mix_mb':  ['/content/drive/Shareddrives/NewsRec/data/cit/common_user_item.csv',
                              '/content/drive/Shareddrives/NewsRec/data/cit/read_articles.csv',
                              '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_mbert.pt',
                              'personId', 'contentId', 100],
               'cit_mix_xlm': ['/content/drive/Shareddrives/NewsRec/data/cit/common_user_item.csv',
                              '/content/drive/Shareddrives/NewsRec/data/cit/read_articles.csv',
                              '/content/drive/Shareddrives/NewsRec/data/cit/feat_tensor_xlm.pt',
                              'personId', 'contentId', 100],


               'mind':        ['/content/drive/Shareddrives/NewsRec/data/mind/MINDlarge_train/common_user_item_0.csv',
                               '/content/drive/Shareddrives/NewsRec/data/mind/MINDlarge_train/read_article_titles_0.csv',
                               '/content/drive/Shareddrives/NewsRec/data/mind/MINDlarge_train/feat_tensor_0.pt',
                               'User_ID', 'News_ID', 3]
               }

##### Funs

In [ ]:
def graph_con(df_users, df_items, user_naming, item_naming, feature_tensor):
    # Create a mapping from unique user indices to range [0, num_user_nodes):
    unique_user_id = df_users[user_naming].unique()
    unique_user_id = pd.DataFrame(data={
        'userId': unique_user_id,
        'mappedID': pd.RangeIndex(len(unique_user_id)),
    })
    print("Mapping of user IDs to consecutive values:")
    print("==========================================")
    print(unique_user_id.head())
    print()
    # Create a mapping from unique item indices to range [0, num_item_nodes):
    assert not any(df_items[item_naming].values != df_items[item_naming].unique())
    unique_item_id = df_items[item_naming]
    unique_item_id = pd.DataFrame(data={
        'itemId': unique_item_id,
        'mappedID': pd.RangeIndex(len(unique_item_id)),
    })
    print("Mapping of item IDs to consecutive values:")
    print("===========================================")
    print(unique_item_id.head())
    # Perform merge to obtain the edges from users and items:
    ratings_user_id = pd.merge(df_users[user_naming], unique_user_id,
                                left_on=user_naming, right_on='userId', how='left')
    ratings_user_id = torch.from_numpy(ratings_user_id['mappedID'].values)
    ratings_item_id = pd.merge(df_users[item_naming], unique_item_id,
                                left_on=item_naming, right_on='itemId', how='left')
    ratings_item_id = torch.from_numpy(ratings_item_id['mappedID'].values)
    # Random edges
    if random_edges:
      ratings_user_id = torch.from_numpy(np.random.randint(low=0, high=len(unique_user_id)-1, size=len(unique_user_id)))
      ratings_item_id = torch.from_numpy(np.random.randint(low=0, high=len(unique_item_id)-1, size=len(unique_user_id)))
    # With this, we are ready to construct our `edge_index` in COO format
    # following PyG semantics:
    edge_index_coo_user_to_item = torch.stack([ratings_user_id, ratings_item_id], dim=0)

    print()
    print("Final edge indices pointing from users to items:")
    print("=================================================")
    print(edge_index_coo_user_to_item)
    data = HeteroData()

    # Save node indices:
    data["user"].node_id = torch.arange(len(unique_user_id))
    data["item"].node_id = torch.arange(len(unique_item_id))

    # Add the node features and edge indices:
    if item_feature_init == 'plm':
      data["item"].x = feature_tensor  # DONE
      if user_has_features:
      # only random init for user features
        data["user"].x = torch.Tensor(np.random.random((len(unique_user_id), user_feature_size)))
        print('user features shape:', data["user"].x.shape)
    elif item_feature_init == 'random':
      data["item"].x = torch.Tensor(np.random.random((len(unique_item_id), item_feature_size)))
    elif item_feature_init == 'zero':
      data["item"].x = torch.Tensor(np.zeros((len(unique_item_id), item_feature_size)))
    else:
      raise 'Need item feature init type: plm, random, zero'

    print('item features shape:', data["item"].x.shape)
    data["user", "rates", "item"].edge_index = edge_index_coo_user_to_item  # DONE

    # We also need to make sure to add the reverse edges from items to users
    # in order to let a GNN be able to pass messages in both directions.
    # We can leverage the `T.ToUndirected()` transform for this from PyG:
    data = T.ToUndirected()(data)
    del data['item', 'rev_rates', 'user'].edge_label  # Remove "reverse" label.

    print(data)
    print("=============================")
    print('user num_nodes:', data["user"].num_nodes)
    print('user num_features:', data["user"].num_features)
    print('item num_nodes:', data["item"].num_nodes)
    print('item num_features:', data["item"].num_features)
    print('num_edges user->item:', data["user", "rates", "item"].num_edges)
    print('num_edges item->user:', data["item", "rev_rates", "user"].num_edges)

    return data

In [ ]:
def data_loader(data):
    # For this, we first split the set of edges into
    # training (80%), validation (10%), and testing edges (10%).
    # Across the training edges, we use 70% of edges for message passing,
    # and 30% of edges for supervision.
    # We further want to generate fixed negative edges for evaluation with a ratio of 2:1.
    # Negative edges during training will be generated on-the-fly.
    # We can leverage the `RandomLinkSplit()` transform for this from PyG:
    transform = T.RandomLinkSplit(
        num_val=0.1,
        num_test=0.1,
        disjoint_train_ratio=0.3,
        neg_sampling_ratio=neg_sampling_ratio,
        add_negative_train_samples=False,       # negative samples generated on-the-fly
        edge_types=("user", "rates", "item"),
        rev_edge_types=("item", "rev_rates", "user"), 
    )


    train_data, val_data, test_data = transform(data)
    print("Training data:")
    print("==============")
    print(train_data)
    print()
    print("Validation data:")
    print("================")
    print(val_data)
    print()
    print("Test data:")
    print("================")
    print(test_data)
    # In the first hop, we sample at most 10 neighbors.
    # In the second hop, we sample at most 5 neighbors.
    # In addition, during training, we want to sample negative edges on-the-fly with
    # a ratio of 2:1.
    # We can make use of the `loader.LinkNeighborLoader` from PyG:
    # Define seed edges:
    edge_label_index = train_data["user", "rates", "item"].edge_label_index
    edge_label = train_data["user", "rates", "item"].edge_label

    train_loader = LinkNeighborLoader(
        data=train_data,
        num_neighbors=num_neighbors,
        neg_sampling_ratio=neg_sampling_ratio,
        edge_label_index=(("user", "rates", "item"), edge_label_index),
        edge_label=edge_label,
        batch_size=batch_size,
        shuffle=True,
    )

    # Inspect a sample:
    sampled_data = next(iter(train_loader))

    print("Sampled Train mini-batch:")
    print("===================")
    print(sampled_data)

    assert sampled_data["user", "rates", "item"].edge_label_index.size(1) == (neg_sampling_ratio + 1) * batch_size # 2 edges (1 pos + 1 neg) * batch_size
    assert sampled_data["user", "rates", "item"].edge_label.min() == 0
    assert sampled_data["user", "rates", "item"].edge_label.max() == 1
    # Define the validation seed edges:
    edge_label_index = val_data["user", "rates", "item"].edge_label_index
    edge_label = val_data["user", "rates", "item"].edge_label

    val_loader = LinkNeighborLoader(
        data=val_data,
        num_neighbors=num_neighbors,
        edge_label_index=(("user", "rates", "item"), edge_label_index),
        edge_label=edge_label,
        batch_size=int((neg_sampling_ratio + 1) * batch_size),
        shuffle=False,
    )

    sampled_data = next(iter(val_loader))

    print("Sampled Validation mini-batch:")
    print("===================")
    print(sampled_data)

    assert sampled_data["user", "rates", "item"].edge_label_index.size(1) == (neg_sampling_ratio + 1) * batch_size
    assert sampled_data["user", "rates", "item"].edge_label.min() >= 0
    assert sampled_data["user", "rates", "item"].edge_label.max() <= 1
    # Define the test seed edges:
    edge_label_index = test_data["user", "rates", "item"].edge_label_index
    edge_label = test_data["user", "rates", "item"].edge_label

    test_loader = LinkNeighborLoader(
        data=test_data,
        num_neighbors=num_neighbors,
        edge_label_index=(("user", "rates", "item"), edge_label_index),
        edge_label=edge_label,
        batch_size=int((neg_sampling_ratio + 1) * batch_size),
        shuffle=False,
    )

    sampled_data = next(iter(test_loader))

    print("Sampled Test mini-batch:")
    print("===================")
    print(sampled_data)

    assert sampled_data["user", "rates", "item"].edge_label_index.size(1) == (neg_sampling_ratio + 1) * batch_size
    assert sampled_data["user", "rates", "item"].edge_label.min() >= 0
    assert sampled_data["user", "rates", "item"].edge_label.max() <= 1

    return train_loader, val_loader, test_loader

In [ ]:
def train(model, train_loader, epochs):
    # training
    train_loss = []
    for epoch in range(1, epochs+1):
        model.train()
        total_loss = total_examples = 0
        for batch_data in tqdm.tqdm(train_loader):
            optimizer.zero_grad()
            batch_data = batch_data.to(device)
            
            pred = model(batch_data) # `forward` pass of the model
            ground_truth = batch_data['user', 'item'].edge_label
            
            loss = criterion(pred, ground_truth)
            loss.backward()
            optimizer.step()
            total_loss += float(loss) * pred.numel()
            total_examples += pred.numel()
        
        epoch_loss = total_loss / total_examples
        train_loss.append(epoch_loss)
        print(f"\nEpoch: {epoch:03d}, Loss: {epoch_loss:.4f}")
    return train_loss

In [ ]:
def test(model, test_loader):
    # test
    preds, ground_truths = [], []
    model.eval()
    for batch_data in tqdm.tqdm(test_loader):
        with torch.no_grad():
            batch_data = batch_data.to(device)
            preds.append(model(batch_data))
            ground_truths.append(batch_data['user', 'item'].edge_label)

    pred = torch.cat(preds, dim=0).cpu().numpy()
    ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
    auc = roc_auc_score(ground_truth, pred)
    print()
    # print(f"Test AUC: {auc:.4f}")

    out = torch.Tensor(pred).sigmoid().cpu().numpy()
    auc = roc_auc_score(ground_truth, out)
    print(f'Test AUC:', auc)

    probs = np.rint(out)
    f1_s = f1_score(ground_truth, probs)
    print(f'Test F1:', f1_s)

    return auc, f1_s

##### Loop

In [ ]:
user_has_features = True
user_feature_size = 32          # 32
item_feature_init = 'plm'       # 'plm', 'zero', 'random'
item_feature_size = 768         # feature_tensor.size()[1]
random_edges = False

neg_sampling_ratio = 1.0        # hyper: 1, 2
batch_size = 256
num_neighbors = [20, 10]         # hyper: [10, 5], [10, 10], [20, 10], [20, 20]

hidden_channels = 32                          # hyper: 16, 32, 64
GNN_Conv_Layer = SAGEConv                     # hyper: SAGEConv, GATConv
remove_embedding_layer = user_has_features    # False

In [ ]:
current = list(datas_dict.keys())
for dataset_name in current:
    cur_dataset = datas_dict[dataset_name]
    fusers = cur_dataset[0]
    fitems = cur_dataset[1]
    user_naming, item_naming = cur_dataset[3], cur_dataset[4]
    df_users, df_items = get_users_and_items(fusers, fitems, user_naming, item_naming)

    ffeat_items = cur_dataset[2]
    feature_tensor = torch.load(ffeat_items, map_location=device)
    print('feature tensor:', feature_tensor.shape)
    assert feature_tensor.shape[0] == df_users[item_naming].nunique()

    data = graph_con(df_users, df_items, user_naming, item_naming, feature_tensor)
    train_loader, val_loader, test_loader = data_loader(data)

    # data contains the whole graph
    num_user_nodes = data["user"].num_nodes
    num_item_nodes = data["item"].num_nodes
    graph_metadata = data.metadata()

    model = Model(user_feature_size, item_feature_size, num_user_nodes, num_item_nodes, hidden_channels, graph_metadata)
    model = model.to(device)
    print(f"Device: '{device}'")
    print(model)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = F.binary_cross_entropy_with_logits

    epochs = cur_dataset[5]
    train_loss = train(model, train_loader, epochs)
    # val_auc, val_f1 = validate(model, val_loader)
    test_auc, test_f1 = test(model, test_loader)

    # all_train_losses.append(train_loss)
    # all_vals.append([val_auc, val_f1])
    # all_tests.append([test_auc, test_f1])

    fpath = f'/content/drive/Shareddrives/NewsRec/data/k_log_{dataset_name}.txt'

    with open(fpath, 'w') as f:
      f.write('train,')
      f.write(','.join(map(str, train_loss)))
      f.write('\ntest,')
      f.write(f',{test_auc},{test_f1}')

users
users shape: (1857389, 2)
unique users: 535259
unique articles: 15721
                                              userId  \
0  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
1  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
2  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
3  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   
4  cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov   

                                         id  
0  01c74ff7ec02862bed5f861cba7e7226dfd31beb  
1  2228c8e09bd615b509f87347675f4f0f1b74439e  
2  265d0e347e08d19ab62ffe4eadd0333eaad6e57c  
3  3011c1cce4999e3c4e02bd887468d1d1d90b7807  
4  44d3e6498035638f2e1c0580332b67e0d71af45c  

items
items shape: (15721, 2)
unique articles: 15721
                                         id  \
0  6a0612e60690288a776834811004ce133f326cee   
1  b40a30877124510cf65683b6c9391d927e20f89d   
2  29b7afc1d16d34b639597cab200c9c4c96e2b69e   
3  e6877439db8ca3bde8be8f6edb741fcc22c2772c   
4  a30fec2f3e9f2e408a2

100%|██████████| 1742/1742 [01:29<00:00, 19.41it/s]



Epoch: 001, Loss: 0.3720


100%|██████████| 1742/1742 [01:25<00:00, 20.40it/s]



Epoch: 002, Loss: 0.2494


...

100%|██████████| 39/39 [00:01<00:00, 19.56it/s]



Epoch: 100, Loss: 0.4465


100%|██████████| 16/16 [00:00<00:00, 19.83it/s]



Test AUC: 0.8433426400879911
Test F1: 0.7658001724775163
users
users shape: (4658157, 2)
unique users: 155855
unique articles: 67147
   User_ID  News_ID
0  U321454  N128643
1  U578952  N122359
2  U578952  N110096
3  U578952   N20583
4  U578952  N128736

items
items shape: (67147, 2)
unique articles: 67147
  News_ID                                              title
0  N88753  The Brands Queen Elizabeth, Prince Charles, an...
1  N45436    Walmart Slashes Prices on Last-Generation iPads
2  N23144                      50 Worst Habits For Belly Fat
3  N86255  Dispose of unwanted prescription drugs during ...
4  N93187  The Cost of Trump's Aid Freeze in the Trenches...

article set is same: True
feature tensor: torch.Size([67147, 768])
Mapping of user IDs to consecutive values:
    userId  mappedID
0  U321454         0
1  U578952         1
2  U100987         2
3   U55226         3
4  U211115         4

Mapping of item IDs to consecutive values:
   itemId  mappedID
0  N88753         0
1  N4

 79%|███████▉  | 3457/4368 [18:10<04:51,  3.13it/s]

# Draft

In [ ]:
batch_data

HeteroData(
  user={ node_id=[224] },
  item={
    node_id=[99],
    x=[99, 768]
  },
  (user, rates, item)={
    edge_index=[2, 205],
    edge_label=[20],
    edge_label_index=[2, 20],
    input_id=[20]
  },
  (item, rev_rates, user)={ edge_index=[2, 150] }
)

What is input id??

In [ ]:
batch_data['user', 'rates', 'item'].input_id

tensor([557056, 557057, 557058, 557059, 557060, 557061, 557062, 557063, 557064,
        557065, 557066, 557067, 557068, 557069, 557070, 557071, 557072, 557073,
        557074, 557075, 557076, 557077, 557078, 557079, 557080, 557081, 557082,
        557083, 557084, 557085, 557086, 557087, 557088, 557089, 557090, 557091,
        557092, 557093, 557094, 557095, 557096, 557097, 557098, 557099, 557100,
        557101, 557102, 557103, 557104, 557105, 557106, 557107, 557108, 557109,
        557110, 557111, 557112, 557113, 557114, 557115, 557116, 557117, 557118,
        557119, 557120, 557121, 557122, 557123, 557124, 557125, 557126, 557127,
        557128, 557129, 557130, 557131, 557132, 557133, 557134, 557135, 557136,
        557137, 557138, 557139, 557140, 557141, 557142, 557143, 557144, 557145,
        557146, 557147, 557148, 557149, 557150, 557151, 557152, 557153, 557154,
        557155, 557156, 557157, 557158, 557159, 557160, 557161, 557162, 557163,
        557164, 557165, 557166, 557167, 

In [ ]:
batch_data['user', 'rates', 'item'].edge_label_index

tensor([[113, 119, 125, 139,   0,  99,  80,  77,  98,  67, 128,  31,  47,  16,
          91,  33,  55,  24,  30,  74,  88, 146, 111,  41, 102,  73, 108,  53,
          60,  28, 137,  23,  38,  87, 100, 110,  97,  65,  51,  34, 118,  19,
         155,  44, 150,  13,  54,  27, 107,  25,  50, 123, 112,  64,  92,  43,
          39,  76,  52, 109, 147,  94,   3,  42,  75,   7, 157, 156,  96,   9,
         142, 129,  71,  22,  46,  10,  81, 122,  37,  78,  21,  11,   5,  69,
          14,  68, 144,  17,  90, 145, 154, 152, 149,  95,  12, 103, 104,  93,
          63, 106,  40,  29, 121, 140,  35,  49,  57, 126, 127,  15,  82,   1,
          59, 117,  89, 124, 120, 132,  85,  79,  45,  56,  20, 151, 114,  26,
          86,  32,  48,   6,  84,  66, 130, 116, 115,   8, 148, 153, 136, 141,
          58, 101,  61,  70, 143,  62,   4,   2, 135, 131,  72,  36, 134, 138,
          18, 133,  83, 105],
        [  2,  80, 100,  92,  25,  38,  74,  85, 144,  99, 113,  84,  17, 131,
          61,  58,   5

In [ ]:
batch_data['user', 'rates', 'item'].edge_label

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')

In [ ]:
ground_truths[-1]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')

In [ ]:
preds[-1]

tensor([ -3.6659,  -5.9209,  -3.8125,  -3.8156,  -4.9753,  -4.2238,  -4.2707,
        -11.7944,  -4.9251,  -3.5596,  -6.4042,  -0.0456,  -1.0130,  -7.3948,
        -13.3436,  -3.7321,  -6.4111,  -6.3421,  -2.7053,   1.7885,  -6.3836,
         -8.3329,  -6.3701,  -5.0134,  -1.7026,  -4.2668, -10.5365, -12.4031,
         -7.9567,  -4.2070,  -3.5657,  -1.4284, -10.8802,  -8.2110,  -6.4754,
         -3.1270,  -8.6796,  -4.9021,  -9.6323,  -8.6613, -10.1006,  -3.5280,
         -2.2421,  -3.4787,  -9.1882,  -2.7364,  -3.8944,  -9.9798,  -5.8470,
         -3.2481,  -8.0756, -11.7022,  -3.6880,  -6.8287,  -6.6260,  -8.0269,
          0.0511, -11.6987,  -0.9760,  -9.0100, -11.5480,  -8.1490,  -8.7247,
         -6.7875,   0.2181,  -7.4815,  -2.0227,  -9.6268,  -3.5110,  -7.6323,
         -1.2116,  -4.2475,  -8.2994,  -5.1759,  -2.0947,  -9.9370,  -4.4411,
         -5.1921,  -3.3492,  -8.0734,  -5.4666,  -3.7695,  -8.5921,  -5.9587,
         -3.9303,  -3.1428,  -4.4850,  -7.2361,  -6.2845,  -5.70

In [ ]:
probs = preds[-1].view(-1).sigmoid().cpu().numpy()
probs = np.rint(probs)
probs

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.], dtype=float32)

In [ ]:
(len(probs) - np.sum(np.rint(probs))) / len(probs)

0.7552625491556789

In [ ]:
# draft
def predictions(max = 1000, threshold = 0.99):
    """
    Creates predictions for the specified run.
    :param run_id: model id
    :param max: the maximum amount of predictions to output
    """
    pred_edges = []

    loader = NeighborLoader(data, num_neighbors = [10] * 2, shuffle = True, input_nodes = None, batch_size = batch_size)
    threshold_tensor = torch.tensor([threshold]).to(device)
    for batch in tqdm(loader):
        batch.to(device)
        z = model.encode(batch.x, batch.edge_index)
        # collecting negative edge tuples ensure that the decode are actual non-existing edges
        neg_edge_index = negative_sampling(edge_index = batch.edge_index, num_nodes = None, num_neg_samples = None, method = 'sparse')
        out = model.decode(z, neg_edge_index).view(-1).sigmoid()
        pred = ((out > threshold_tensor).float() * 1).cpu().numpy()
        found = np.argwhere(pred == 1)
        if found.size > 0:
            edge_tuples = neg_edge_index.t().cpu().numpy()
            select_index = found.reshape(1, found.size)[0]
            edges = edge_tuples[select_index]
            pred_edges += edges.tolist()
            if len(pred_edges) >= max:
                break
    
    return pd.DataFrame.from_dict([{'source': a, 'target': b} for a,b in pred_edges])